# WAYS demo #1

In this demo, we load presidential polling dataset and generate several plots including a choropleth... using the `altair` package in Python. We then use the WAYS package, to add meta-visualisation to the plots and interactive widgets.

### Install requirements

In [1]:
%%capture
# capture suppresses output of the below:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install -r requirements.txt

### Import WAYS and required packages for this notebook

In [2]:
import geopandas as gpd 
import pandas as pd
import altair as alt

# Import WAYS
import sys; sys.path.insert(0, '..')
from ways_py.ways import (meta_hist, altair_bin_jupyter_widgets, altair_scale_jupyter_widgets,
                          get_altair_color_obj)

In [3]:
geo_states = gpd.read_file('choropleth_teaching/gz_2010_us_040_00_500k.json')
df_polls = pd.read_csv('choropleth_teaching/presidential_poll_averages_2020.csv')

In [4]:
df_polls.head()

,cycle,state,modeldate,candidate_name,pct_estimate,pct_trend_adjusted
0,2020,Wyoming,11/03/2020,Joseph R. Biden Jr.,30.81486,30.82599
1,2020,Wisconsin,11/03/2020,Joseph R. Biden Jr.,52.12642,52.09584
2,2020,West Virginia,11/03/2020,Joseph R. Biden Jr.,33.49125,33.51517
3,2020,Washington,11/03/2020,Joseph R. Biden Jr.,59.34201,59.39408
4,2020,Virginia,11/03/2020,Joseph R. Biden Jr.,53.74120,53.72101


On how many dates was the polling done?

In [5]:
len(df_polls.modeldate.unique())

251

Filter our poll data to remove third party candidates.

In [6]:
df_polls = df_polls[
    (df_polls.candidate_name == 'Donald Trump') |
    (df_polls.candidate_name == 'Joseph R. Biden Jr.')
]

The geo_states variable has polygons for each state.

In [7]:
geo_states.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ..."
1,0400000US25,25,Massachusetts,,7800.058,"MULTIPOLYGON (((-70.83204 41.60650, -70.82373 ..."
2,0400000US26,26,Michigan,,56538.901,"MULTIPOLYGON (((-88.68443 48.11579, -88.67563 ..."
3,0400000US30,30,Montana,,145545.801,"POLYGON ((-104.05770 44.99743, -104.25015 44.9..."
4,0400000US32,32,Nevada,,109781.180,"POLYGON ((-114.05060 37.00040, -114.04999 36.9..."


In [8]:
trump_data = df_polls[
    df_polls.candidate_name == 'Donald Trump'
]

biden_data = df_polls[
    df_polls.candidate_name == 'Joseph R. Biden Jr.'
]

Our spatial and poll data have the name of the state in common. We will change the name of the state to NAME to match our geospatial dataframe.

In [9]:
trump_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']
biden_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']

We can join the geospatial and poll data using the NAME column (the name of the state).

In [10]:
# Create separate date frame for trump and biden
# Add the poll data
geo_states_trump = geo_states.merge(trump_data, on='NAME')
geo_states_biden = geo_states.merge(biden_data, on='NAME')

In [11]:
(geo_states_trump.modeldate == '11/03/2020').value_counts()

False    9972
True       51
Name: modeldate, dtype: int64

In [12]:
geo_states_biden.head()

,GEO_ID,STATE,NAME,LSAD,CENSUSAREA,geometry,cycle,modeldate,candidate_name,pct_estimate,pct_trend_adjusted
0,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/03/2020,Joseph R. Biden Jr.,53.31518,53.32106
1,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/02/2020,Joseph R. Biden Jr.,53.31518,53.32106
2,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,11/01/2020,Joseph R. Biden Jr.,53.70696,53.91672
3,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,10/31/2020,Joseph R. Biden Jr.,53.30098,53.55544
4,0400000US23,23,Maine,,30842.923,"MULTIPOLYGON (((-67.61976 44.51975, -67.61541 ...",2020,10/30/2020,Joseph R. Biden Jr.,53.34166,53.64551


### Create choropleth plot function, decorated with meta histogram

In [13]:
@meta_hist # decorators modify the output of usa_choro when called to be more than just the return
def usa_choro(candidate_geo_states, color, title):
    """Plot a choropleth of the US states with the candidate vote percentage mapped to the color;
    pass in altair color object and a title"""
    chart = alt.Chart(candidate_geo_states, title=title).mark_geoshape()
    chart = chart.encode(
        color,
        tooltip=['NAME', 'pct_estimate']
    ).properties(
        width=500,
        height=300
    ).project(
        type='albersUsa'
    )
    return chart

## Adding interactivity with Jupyter interact widgets

Here the user working on the the US presidential poll choropleth visualisation has options related to the colour-binning controlled via widgets defined in WAYS.

In [14]:
from ipywidgets import widgets

### Create widgets to control which parts of the data to visualise

In [15]:
import datetime

# Simple dropdown to switch between Biden and Trump's data
candidate = widgets.Dropdown(value='Biden', options=['Trump', 'Biden'], description = 'Candidate')

# Get an ordered list of the dates (as strings) on which polling occured
unsorted_datestrings = list(set(list(geo_states_biden['modeldate'])))
dates = sorted(unsorted_datestrings, key=lambda x: datetime.datetime.strptime(x, '%m/%d/%Y'))

# Choose the polling date to visualise
date = widgets.SelectionSlider(value='10/03/2020', options=dates, description='Date', continuous_update=False)

# create dictionary for widgets (we'll need this later)
data_widgets = {
        'candidate': candidate,
        'date': date
}
# ... and a list with these widgets to use for the grid
data_widgets_list = list(data_widgets.values())

# Plot does not automatically get generated, only the widgets are visible until this (or anything else) clicked
generate_button = widgets.Button(description="Generate Plot")
def on_button_clicked(b):
    date.value='11/03/2020' # This will get the plot to generate by changing one of the widgets
generate_button.on_click(on_button_clicked)
data_widgets_list.insert(0, generate_button) # Arrange it so this button is at the front

# Create the GridBox to arrange the widgets into rows of three

data_widgets_grid = widgets.GridBox(data_widgets_list,
                                    layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

### Load WAYS widgets that will control parameters of the altair plot

In [16]:
# get the widgets from WAYS
bin_widgets = altair_bin_jupyter_widgets()

scale_widgets = altair_scale_jupyter_widgets()

### Define interactive version of usa_choro 

In [17]:
def interact_usa_choro(**kwargs):

    # Chooses the candidate dataset
    candidate_geo_states = geo_states_biden
    if candidate.value == 'Trump':
        candidate_geo_states = geo_states_trump

    # Choose which polling date to display
    candidate_geo_states = candidate_geo_states[
        (candidate_geo_states.modeldate == date.value)
    ]

    # Give the choropleth plot a title
    title = 'Poll estimate for ' + candidate.value + ' on ' + date.value
    
    # Choose column of df to plot
    column = "pct_estimate"

    # Use the WAYS widgets to generate the altair color object
    color = get_altair_color_obj(bin_widgets, scale_widgets, column)
    
    # Pass the data, color object and title into the choropleth
    display(usa_choro(candidate_geo_states, color, title))

### Pass the values of widgets into the args of the interactive choropleth

In [18]:
# Get a dictionary of the widgets to use as controls
controls = data_widgets | bin_widgets | scale_widgets
controls.pop('bin_grid') # Remove the grid widgets
controls.pop('scale_grid')

# Use Jupyter widgets interactive_output to apply the control widgets to the interactive plot
interactive_choropleth = widgets.interactive_output(
    interact_usa_choro,
    controls
)

### Display the interactive plot with the widgets that control it

In [19]:
display(data_widgets_grid,
        bin_widgets['bin_grid'],
        scale_widgets['scale_grid'],
        interactive_choropleth)

GridBox(children=(Button(description='Generate Plot', style=ButtonStyle()), Dropdown(description='Candidate', …

GridBox(children=(Checkbox(value=True, description='Bin'), IntSlider(value=100, continuous_update=False, descr…

GridBox(children=(RadioButtons(description='Color Method', options=('Scheme', 'Range'), value='Scheme'), Dropd…

Output()

Notes:
1. Setting the scale to `log` doesn't work unless `bin` is **unchecked**
2. Because the `extent` data range plugs into the `alt.Bin` object, this will only work if `bin` is **checked**

TODO:
1. Add a way of choosing the number of colors for the color range